In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)
print(dataset)

                                                Review  Liked
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]


In [3]:
dataset = dataset[['Review','Liked']]

In [4]:
dataset['Review'] = dataset['Review'].apply(lambda x: x.lower())
dataset['Review'] = dataset['Review'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(dataset[ dataset['Liked'] == 1].size)
print(dataset[ dataset['Liked'] == 0].size)

for idx,row in dataset.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(dataset['Review'].values)
X = tokenizer.texts_to_sequences(dataset['Review'].values)
X = pad_sequences(X)

1000
1000


In [5]:
embed_dim = 256
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 256)           512000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 32, 256)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               355152    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 867,546
Trainable params: 867,546
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
Y = pd.get_dummies(dataset['Liked']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 123)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(670, 32) (670, 2)
(330, 32) (330, 2)


In [7]:
batch_size = 16
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
42/42 - 26s - loss: 0.6868 - accuracy: 0.5672
Epoch 2/10
42/42 - 4s - loss: 0.5156 - accuracy: 0.7910
Epoch 3/10
42/42 - 3s - loss: 0.2501 - accuracy: 0.9149
Epoch 4/10
42/42 - 4s - loss: 0.1203 - accuracy: 0.9567
Epoch 5/10
42/42 - 4s - loss: 0.1177 - accuracy: 0.9612
Epoch 6/10
42/42 - 4s - loss: 0.0422 - accuracy: 0.9925
Epoch 7/10
42/42 - 3s - loss: 0.0246 - accuracy: 0.9940
Epoch 8/10
42/42 - 4s - loss: 0.0119 - accuracy: 0.9985
Epoch 9/10
42/42 - 3s - loss: 0.0100 - accuracy: 0.9970
Epoch 10/10
42/42 - 3s - loss: 0.0034 - accuracy: 1.0000


In [8]:
validation_size = 200

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

9/9 - 1s - loss: 0.7812 - accuracy: 0.7923
score: 0.78
acc: 0.79
